In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
d = os.path.dirname(os.getcwd())
sys.path.insert(0, d)

In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision
from torchvision.datasets import CIFAR10, MNIST
from torchvision.transforms import transforms

from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from utils.attacks import fgsm, pgd
from utils.utils import plot_images
from utils.train_utils import eval_epoch, train_model
from models.resnet import ResNet, SparseResNet
from models.resnet_FReLU import FResNet, FSparseResNet

1.7.0
True


In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [4]:
tr_transform = transforms.Compose([
#     transforms.RandomCrop(32, padding=4, padding_mode='reflect'),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
vl_transform = transforms.Compose([
    transforms.ToTensor()
])

ds = MNIST('../data', train=True, download=True, transform=tr_transform)
ds_test = MNIST('../data', train=False, download=True, transform=vl_transform)

batch_size = 200
train_dl = DataLoader(ds, batch_size, shuffle=True)
valid_dl = DataLoader(ds_test, batch_size, shuffle=True)

for X, y in valid_dl:
    X, y = X.to(device), y.to(device)
    break

In [5]:
import torch
import torch.nn as nn

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

class FlattenReLU(torch.autograd.Function):
    @staticmethod
    def forward(self, x, thresholds=None):
        self.save_for_backward(x, thresholds)
        x = torch.where(x > thresholds, thresholds, x) 
        x[x<0] = 0
        return x

    @staticmethod
    def backward(self, grad_output):
        x, thresholds = self.saved_tensors
        x_grad = grad_output.clone()
        x_grad[x>thresholds] = 0
        x_grad[x<0] = 0
        return x_grad, None

FRelu = FlattenReLU.apply

In [6]:
class ML(nn.Linear):

    def __init__(self, in_features, out_features, bias=False):
        super().__init__(in_features, out_features, bias)

    def forward(self, x):
        xe = torch.stack([x]*self.weight.size(0), dim=1)
        we = torch.stack([self.weight]*x.size(0), dim=0)
        z, _ = (we * xe).topk(int(0.1*self.weight.size(1)))
        thresholds = z.sum(dim=2)
        x = F.linear(x, self.weight, self.bias)
        return FRelu(x, thresholds)


In [7]:
class Net1(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = ML(784, 200, bias=False)
        self.fc2 = ML(200, 10, bias=False)
        
    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [8]:
model = Net1().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', min_lr=1e-7, patience=2, verbose=True)

train_model(model, train_dl, valid_dl, optimizer, loss_fn, scheduler, epochs=30, device=device)

Epoch: 1 Validation Loss: 0.8033 accuracy: 0.8428, time: 0:00:20
Epoch: 2 Validation Loss: 0.4769 accuracy: 0.8662, time: 0:00:19
Epoch: 3 Validation Loss: 0.4047 accuracy: 0.8704, time: 0:00:17
Epoch: 4 Validation Loss: 0.3636 accuracy: 0.8760, time: 0:00:17
Epoch: 5 Validation Loss: 0.3407 accuracy: 0.8823, time: 0:00:16
Epoch: 6 Validation Loss: 0.3416 accuracy: 0.8818, time: 0:00:18
epochs_no_improve: 1/4
Epoch: 7 Validation Loss: 0.3211 accuracy: 0.8821, time: 0:00:16
Epoch: 8 Validation Loss: 0.3220 accuracy: 0.8838, time: 0:00:17
epochs_no_improve: 1/4
Epoch: 9 Validation Loss: 0.3177 accuracy: 0.8853, time: 0:00:17
Epoch: 10 Validation Loss: 0.3071 accuracy: 0.8866, time: 0:00:17
Epoch: 11 Validation Loss: 0.3115 accuracy: 0.8862, time: 0:00:17
epochs_no_improve: 1/4
Epoch: 12 Validation Loss: 0.3039 accuracy: 0.8880, time: 0:00:16
Epoch: 13 Validation Loss: 0.3087 accuracy: 0.8875, time: 0:00:17
epochs_no_improve: 1/4
Epoch: 14 Validation Loss: 0.3081 accuracy: 0.8867, time: 0

In [556]:
class myLinear(nn.Linear):

    def __init__(self, in_features, out_features, bias=False):
        super().__init__(in_features, out_features, bias)

    def forward(self, x):
        xe = torch.stack([x]*self.weight.size(0), dim=1)
        we = torch.stack([self.weight]*x.size(0), dim=0)
        z, _ = (we * xe).topk(int(0.3*self.weight.size(1)))
        thresholds = z.sum(dim=2)
        x = F.linear(x, self.weight, self.bias)    
        return FRelu(x, thresholds)


In [557]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = myLinear(10, 2, bias=False)
        
    def forward(self, x):
        x = self.fc1(x)
        return x

model = Net()
model

Net(
  (fc1): myLinear(in_features=10, out_features=2, bias=False)
)

In [558]:
w = model.fc1.weight.clone()
w = torch.Tensor([[1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1], 
                  [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
                 ])
x = torch.Tensor([[1,2,3,4,5,6,7,8,9,10], [10,9,-8,-7,-6,-5,4,3,2,1], [6,7,8,9,10,5,4,3,2,1]])
model.fc1.weight.data = w
print(model.fc1.weight.data)
print(x)
y = torch.Tensor([1,0,0])

tensor([[1.0000, 0.9000, 0.8000, 0.7000, 0.6000, 0.5000, 0.4000, 0.3000, 0.2000,
         0.1000],
        [0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000, 0.7000, 0.8000, 0.9000,
         1.0000]])
tensor([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        [10.,  9., -8., -7., -6., -5.,  4.,  3.,  2.,  1.],
        [ 6.,  7.,  8.,  9., 10.,  5.,  4.,  3.,  2.,  1.]])


In [561]:
xe = torch.stack([x]*w.size(0), dim=1)
xe

tensor([[[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
         [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]],

        [[10.,  9., -8., -7., -6., -5.,  4.,  3.,  2.,  1.],
         [10.,  9., -8., -7., -6., -5.,  4.,  3.,  2.,  1.]],

        [[ 6.,  7.,  8.,  9., 10.,  5.,  4.,  3.,  2.,  1.],
         [ 6.,  7.,  8.,  9., 10.,  5.,  4.,  3.,  2.,  1.]]])

In [562]:
we = torch.stack([w]*x.size(0), dim=0)
we

tensor([[[1.0000, 0.9000, 0.8000, 0.7000, 0.6000, 0.5000, 0.4000, 0.3000,
          0.2000, 0.1000],
         [0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000, 0.7000, 0.8000,
          0.9000, 1.0000]],

        [[1.0000, 0.9000, 0.8000, 0.7000, 0.6000, 0.5000, 0.4000, 0.3000,
          0.2000, 0.1000],
         [0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000, 0.7000, 0.8000,
          0.9000, 1.0000]],

        [[1.0000, 0.9000, 0.8000, 0.7000, 0.6000, 0.5000, 0.4000, 0.3000,
          0.2000, 0.1000],
         [0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000, 0.7000, 0.8000,
          0.9000, 1.0000]]])

In [563]:
(we * xe)

tensor([[[ 1.0000,  1.8000,  2.4000,  2.8000,  3.0000,  3.0000,  2.8000,
           2.4000,  1.8000,  1.0000],
         [ 0.1000,  0.4000,  0.9000,  1.6000,  2.5000,  3.6000,  4.9000,
           6.4000,  8.1000, 10.0000]],

        [[10.0000,  8.1000, -6.4000, -4.9000, -3.6000, -2.5000,  1.6000,
           0.9000,  0.4000,  0.1000],
         [ 1.0000,  1.8000, -2.4000, -2.8000, -3.0000, -3.0000,  2.8000,
           2.4000,  1.8000,  1.0000]],

        [[ 6.0000,  6.3000,  6.4000,  6.3000,  6.0000,  2.5000,  1.6000,
           0.9000,  0.4000,  0.1000],
         [ 0.6000,  1.4000,  2.4000,  3.6000,  5.0000,  3.0000,  2.8000,
           2.4000,  1.8000,  1.0000]]])

In [570]:
val, _ = (we * xe).topk(int(0.3*w.size(1)))
print(val)

tensor([[[ 3.0000,  3.0000,  2.8000],
         [10.0000,  8.1000,  6.4000]],

        [[10.0000,  8.1000,  1.6000],
         [ 2.8000,  2.4000,  1.8000]],

        [[ 6.4000,  6.3000,  6.3000],
         [ 5.0000,  3.6000,  3.0000]]])


In [571]:
thresholds = val.sum(dim=-1)
print(thresholds)

tensor([[ 8.8000, 24.5000],
        [19.7000,  7.0000],
        [19.0000, 11.6000]])


In [572]:
res = x @ w.T
f = torch.where(res > thresholds, thresholds, res)
f[f<0]=0
f

tensor([[ 8.8000, 24.5000],
        [ 3.7000,  0.0000],
        [19.0000, 11.6000]])

In [573]:
model(x)

tensor([[ 8.8000, 24.5000],
        [ 3.7000,  0.0000],
        [19.0000, 11.6000]], grad_fn=<FlattenReLUBackward>)

In [574]:
x @ w.T

tensor([[22.0000, 38.5000],
        [ 3.7000, -0.4000],
        [36.5000, 24.0000]])